<a href="https://colab.research.google.com/github/bigboyfreezy/recommender-machinelearning/blob/main/contentbasedrecommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Content Based -- content based recommender system using few lines of code and some domain knowledge about machine learning and algebra.
!pip install sentence_transformers
import numpy as np
from sentence_transformers import SentenceTransformer
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.3 MB/s 
     |████████████████████████████████| 4.4 MB 10.1 MB/s 
     |████████████████████████████████| 1.2 MB 44.3 MB/s 
     |████████████████████████████████| 101 kB 11.3 MB/s 
     |████████████████████████████████| 596 kB 60.1 MB/s 
     |████████████████████████████████| 6.6 MB 43.7 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=f25818346fc5853641e8bef4a56b43fd4bca7eefaead07379b0a40987e4b2c5d
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
data = pd.read_csv('https://modcom.co.ke/datasets/imdb_top_1000.csv')
data.head(4)

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"


In [ ]:
#Data Viewing
data = pd.read_csv('https://modcom.co.ke/datasets/imdb_top_1000.csv')
X = np.array(data.Overview)
data.head(10)

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"
5,https://m.media-amazon.com/images/M/MV5BNzA5ZD...,The Lord of the Rings: The Return of the King,2003,U,201 min,"Action, Adventure, Drama",8.9,Gandalf and Aragorn lead the World of Men agai...,94.0,Peter Jackson,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom,1642758,"377,845,905"
6,https://m.media-amazon.com/images/M/MV5BNGNhMD...,Pulp Fiction,1994,A,154 min,"Crime, Drama",8.9,"The lives of two mob hitmen, a boxer, a gangst...",94.0,Quentin Tarantino,John Travolta,Uma Thurman,Samuel L. Jackson,Bruce Willis,1826188,"107,928,762"
7,https://m.media-amazon.com/images/M/MV5BNDE4OT...,Schindler's List,1993,A,195 min,"Biography, Drama, History",8.9,"In German-occupied Poland during World War II,...",94.0,Steven Spielberg,Liam Neeson,Ralph Fiennes,Ben Kingsley,Caroline Goodall,1213505,"96,898,818"
8,https://m.media-amazon.com/images/M/MV5BMjAxMz...,Inception,2010,UA,148 min,"Action, Adventure, Sci-Fi",8.8,A thief who steals corporate secrets through t...,74.0,Christopher Nolan,Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot Page,Ken Watanabe,2067042,"292,576,195"
9,https://m.media-amazon.com/images/M/MV5BMmEzNT...,Fight Club,1999,A,139 min,Drama,8.8,An insomniac office worker and a devil-may-car...,66.0,David Fincher,Brad Pitt,Edward Norton,Meat Loaf,Zach Grenier,1854740,"37,030,102"


When working with textual data the first thing to do is to convert this text into
numbers. More precisely we are converting a string of text into a vector.
In our case we will use the vectorize test to find the similarity between two
vectors. Our recommendation will be the (5) most similar vectors with the one
we are considering.

1. Use BERT to convert our text into a vector
2. Get the cosine similarity (the cosine of the angle between the two
vectors) of a fixed movie (vector) and all the other ones
3. Pick the movies (vectors) with the largest cosine similarity. We are
going to pick 5 of them.

In [ ]:
#Here’s how you convert a text to a vector
text_data = X
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
embeddings = model.encode(text_data,show_progress_bar=True)
embeddings

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/450 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

array([[-0.0823074 , -0.45463464,  1.3678789 , ...,  0.43470237,
         0.15992217, -1.4741831 ],
       [-1.020697  , -1.0519216 ,  0.16398335, ..., -0.2743365 ,
         0.5102986 , -1.118123  ],
       [-0.90921736, -0.5190276 ,  0.49628302, ...,  0.08940516,
         0.5005488 , -1.0971556 ],
       ...,
       [-0.32258818, -0.6107888 , -0.27135688, ..., -0.5176849 ,
         0.8003068 , -0.7838158 ],
       [-0.30706   , -0.8501256 , -0.37546742, ..., -0.26810485,
         0.41472954, -0.728241  ],
       [-0.7734078 , -0.42991334, -0.05513107, ...,  0.04384438,
         0.4410193 , -0.19162664]], dtype=float32)

In [ ]:
cos_sim_data = pd.DataFrame(cosine_similarity(embeddings))
cos_sim_data

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,0.550392,0.350035,0.421647,0.463879,0.408423,0.515788,0.353916,0.387551,0.317381,...,0.305639,0.366211,0.417201,0.488904,0.500020,0.372191,0.415689,0.435626,0.387703,0.394108
1,0.550392,1.000000,0.500683,0.786400,0.515959,0.494177,0.590944,0.616865,0.669451,0.522402,...,0.572684,0.527550,0.541616,0.561910,0.610282,0.562431,0.462835,0.591016,0.463929,0.667192
2,0.350035,0.500683,1.000000,0.558546,0.485291,0.561352,0.590945,0.544381,0.676872,0.625679,...,0.661857,0.611165,0.603801,0.519504,0.566178,0.445671,0.289652,0.492818,0.425312,0.695630
3,0.421647,0.786400,0.558546,1.000000,0.406723,0.457717,0.692699,0.603844,0.589853,0.588154,...,0.600036,0.524070,0.504667,0.470645,0.616529,0.598617,0.459036,0.541068,0.424762,0.571338
4,0.463879,0.515959,0.485291,0.406723,1.000000,0.492655,0.455883,0.449242,0.564627,0.465197,...,0.435063,0.444267,0.510465,0.526206,0.521250,0.410175,0.308236,0.530084,0.403962,0.613202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.372191,0.562431,0.445671,0.598617,0.410175,0.247875,0.504041,0.528951,0.522887,0.526840,...,0.403573,0.463685,0.461474,0.568742,0.617731,1.000000,0.290021,0.626848,0.394188,0.555140
996,0.415689,0.462835,0.289652,0.459036,0.308236,0.368696,0.568177,0.396030,0.323748,0.351758,...,0.564020,0.374122,0.374286,0.461643,0.425178,0.290021,1.000000,0.418450,0.358423,0.317991
997,0.435626,0.591016,0.492818,0.541068,0.530084,0.438132,0.597036,0.573586,0.574997,0.541618,...,0.587393,0.519467,0.467826,0.600152,0.681106,0.626848,0.418450,1.000000,0.623554,0.574718
998,0.387703,0.463929,0.425312,0.424762,0.403962,0.478383,0.565744,0.569549,0.522156,0.513807,...,0.550375,0.575349,0.347166,0.529319,0.477487,0.394188,0.358423,0.623554,1.000000,0.486595


In [ ]:
def give_recommendations(index,print_recommendation = False,print_recommendation_plots= False,print_genres =False):
  index_recomm =cos_sim_data.loc[index].sort_values(ascending=False).index.tolist()[1:6]
  movies_recomm =  data['Series_Title'].loc[index_recomm].values
  result = {'Movies':movies_recomm,'Index':index_recomm}
  if print_recommendation==True:
    print('The watched movie is this one: %s \n'%(data['Series_Title'].loc[index]))
    k=1
    for movie in movies_recomm:
      print('The number %i recommended movie is this one: %s \n'%(k,movie))
      k=k+1
  if print_recommendation_plots==True:
    print('The plot of the watched movie is this one:\n %s \n'%(data['Overview'].loc[index]))
    k=1
    for q in range(len(movies_recomm)):
      plot_q = data['Overview'].loc[index_recomm[q]]
      print('The plot of the number %i recommended movie is this one:\n %s \n'%(k,plot_q))
      k=k+1
  if print_genres==True:
    print('The genres of the watched movie is this one:\n %s \n'%(data['Genre'].loc[index]))
    k=1
    for q in range(len(movies_recomm)):
      plot_q = data['Genre'].loc[index_recomm[q]]
      print('The plot of the number %i recommended movie is this one:\n %s \n'%(k,plot_q))
      k=k+1
  return result

In [ ]:
give_recommendations(6,False,True)

The plot of the watched movie is this one:
 The lives of two mob hitmen, a boxer, a gangster and his wife, and a pair of diner bandits intertwine in four tales of violence and redemption. 

The plot of the number 1 recommended movie is this one:
 A tale of greed, deception, money, power, and murder occur between two best friends: a mafia enforcer and a casino executive compete against each other over a gambling empire, and over a fast-living and fast-loving socialite. 

The plot of the number 2 recommended movie is this one:
 The story of Henry Hill and his life in the mob, covering his relationship with his wife Karen Hill and his mob partners Jimmy Conway and Tommy DeVito in the Italian-American crime syndicate. 

The plot of the number 3 recommended movie is this one:
 After his family is murdered by a notorious and ruthless bandit, a former police officer enlists the services of two outlaws to capture the bandit. 

The plot of the number 4 recommended movie is this one:
 A gang of 

{'Index': [165, 15, 177, 335, 710],
 'Movies': array(['Casino', 'Goodfellas', 'Sholay', 'Special Chabbis', 'Key Largo'],
       dtype=object)}